In [5]:
import pandas as pd
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import warnings
import joblib
warnings.filterwarnings("ignore")


data = pd.read_csv("KACities.csv",header=0)
data.head()

data.info()

df1 = data.copy()
df1['co']=df1['co'].fillna((df1['co'].median()))
df1['no']=df1['no'].fillna((df1['no'].median()))
df1['no2']=df1['no2'].fillna((df1['no2'].median()))
df1['o3']=df1['o3'].fillna((df1['o3'].median()))
df1['so2']=df1['so2'].fillna((df1['so2'].median()))
df1['pm2_5']=df1['pm2_5'].fillna((df1['pm2_5'].median()))
df1['pm10']=df1['pm10'].fillna((df1['pm10'].median()))
df1['nh3']=df1['nh3'].fillna((df1['nh3'].median()))

df1.head()


encoder = LabelEncoder()
df1["city_encoded"] = encoder.fit_transform(df1["city"])
df1.head()

categorical_attributes = list(df1.select_dtypes(include=['object']).columns)
print("categorical_attributes",categorical_attributes)
le=LabelEncoder()
df1['aqi']=le.fit_transform(df1['aqi'].astype(float))
df1.info()

y = df1["aqi"]
x = df1[['co','no','no2','o3','so2','pm2_5','pm10','nh3','city_encoded']]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)


print('\n\n\n\nClasses and number of values in trainset',Counter(y_train))


oversample = SMOTE()
X_train,y_train = oversample.fit_resample(X_train,y_train)
print('\n\n\nClasses and number of values in trainset after SMOTE:',Counter(y_train))


print("\n\nXGBoost Classifier : \n\n")

gbc=XGBClassifier(learning_rate =0.01,n_estimators=100,max_depth=1,
                  min_child_weight=6,subsample=0.8,seed=13)
gbc.fit(X_train,y_train)


pred = gbc.predict(X_test)
xgb_conf_matrix = confusion_matrix(y_test, pred)
accuracy = accuracy_score(y_test, pred) * 100
print("\tConfusion Matrix\n")
print(xgb_conf_matrix)
print("\n")
print("\tAccuracy: ", accuracy)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124374 entries, 0 to 124373
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    124374 non-null  int64  
 1   aqi     124374 non-null  int64  
 2   co      124374 non-null  float64
 3   no      124374 non-null  float64
 4   no2     124374 non-null  float64
 5   o3      124374 non-null  float64
 6   so2     124374 non-null  float64
 7   pm2_5   124374 non-null  float64
 8   pm10    124374 non-null  float64
 9   nh3     124374 non-null  float64
 10  city    124374 non-null  object 
dtypes: float64(8), int64(2), object(1)
memory usage: 10.4+ MB
categorical_attributes ['city']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124374 entries, 0 to 124373
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   date          124374 non-null  int64  
 1   aqi           124374 non-null  int64  
 2   co      